In [11]:
import pandas as pd
import numpy as np
import pickle
import os
from itertools import combinations

import sys
sys.path.append('..')


# PLOTTING
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# CLASSIFICATION
from glmnet import LogitNet
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.decomposition import PCA
from sklearn.metrics import log_loss

from schemata.classification import get_pca_transformed_data, Classifier

# MULTI PROCESSING
import multiprocessing
from functools import partial

## Morphometric statistics

In [9]:
m = LogitNet(random_state=17, **{"alpha":0.5, "standardize":0})
types = pd.DataFrame.from_csv('./data/types.csv').sort_values('c_num')
def classify_morphometry(j,comb=('BC','DBC'), part='axon'):
    path = './data/'+part+'/'

    #get data 
    morph_idx = ['c_num', 'branch_points', 'tips', 'height', 'width', 'depth', 
                 'stems', 'avg_thickness', 'max_thickness', 'total_length', 
                 'total_surface', 'total_volume', 'max_path_dist_to_soma', 
                 'max_branch_order', 'max_segment_path_length', 'median_intermediate_segment_pl', 
                 'median_terminal_segment_pl', 'median_path_angle', 
                 'max_path_angle', 'median_tortuosity', 'max_tortuosity', 
                 'min_branch_angle', 'mean_branch_angle', 'max_branch_angle', 'max_degree', 'tree_asymmetry']
    
    data = pd.read_csv(path+'/morphometry_truncated_%0.2f.csv'%(j*0.1))
    data.sort_values('c_num')
    data = data[morph_idx]
    idx = (types['type'] == comb[0]) | (types['type'] == comb[1])
    data = data[idx.values]
   
    y = types[idx]['type'].values
    y_shuffled = copy.copy(y)
    np.random.shuffle(y_shuffled)

    X = data.set_index(['c_num']).values
    pca = PCA(copy=True, whiten=False)
    kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=17)
    runs = []
    k = 1
    fm_lengths = np.array([0] + [1]*X.shape[1])
    for train_ix, test_ix in kf.split(X, y):

        X_train = X[train_ix]
        y_train = y[train_ix]
        X_test = X[test_ix]
        y_test = y[test_ix]

        X_train, X_test = get_pca_transformed_data(X_train, X_test, pca, fm_lengths)
        r_ = dict(truncated=j*10)
        r_['run_id'] = k


        m.fit(X_train, y_train)
        
        r_['accuracy_train'] = m.score(X_train, y_train)
        r_['accuracy_test'] = m.score(X_test, y_test)
        r_['log_loss_train'] = log_loss(y_train, m.predict_proba(X_train), labels=np.unique(y_train))
        r_['log_loss_test'] = log_loss(y_test, m.predict_proba(X_test), labels=np.unique(y_test))
        r_['shuffled'] = False
        
        runs.append(r_)
        k += 1
        
    #redo on shuffled labels
    k=1
    for train_ix, test_ix in kf.split(X, y_shuffled):

        X_train = X[train_ix]
        y_train = y_shuffled[train_ix]
        X_test = X[test_ix]
        y_test = y_shuffled[test_ix]

        X_train, X_test = get_pca_transformed_data(X_train, X_test, pca, fm_lengths)
        r_ = dict(truncated=j*10)
        r_['run_id'] = k


        m.fit(X_train, y_train)
        
        r_['accuracy_train'] = m.score(X_train, y_train)
        r_['accuracy_test'] = m.score(X_test, y_test)
        r_['log_loss_train'] = log_loss(y_train, m.predict_proba(X_train), labels=np.unique(y_train))
        r_['log_loss_test'] = log_loss(y_test, m.predict_proba(X_test), labels=np.unique(y_test))
        r_['shuffled'] = True
        
        runs.append(r_)
        k += 1

    classification = pd.DataFrame(runs)
    save_path = './results/'+part+'/'+comb[0]+' vs '+comb[1]+'/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    classification.to_csv(save_path+'classification_morphometry_truncated_%0.2f.csv'%(j*0.1))
    
    # train on entire data set and save the model and data
    X_, _ = get_pca_transformed_data(X,X,pca,fm_lengths)
    m.fit(X_,y)
    
    #open(save_path+'classification_morphometry_truncated_%0.2f_model'%(j*0.1),'wb').write(pickle.dumps(m))

In [12]:
TYPES = ['BC', 'BPC', 'BTC', 'ChC', 'DBC', 'MC', 'NGC']
part='full'
truncations = np.array(range(0,10))

### Classification

In [18]:
for c in combinations(TYPES,2):
    with multiprocessing.Pool(10) as pool:
        results = pool.map(partial(classify_morphometry,comb=c,part='full'),truncations)

# Persistence

In [4]:
from scipy.stats import norm, gaussian_kde
def get_persistence_image(data, dim=2, t=1, bins=100, xmax=None, xmin=None,  ymax=None, ymin=None):

    if not data.empty:
        if dim == 1:
            y = np.zeros((bins,))
            if xmax is None:
                xmax = np.max(data['birth'])
            if xmin is None:
                xmin = np.min(data['birth'])
            x = np.linspace(xmin, xmax, bins)

            for k, p in data.iterrows():
                m = np.abs(p['birth'] - p['death'])
                y += m * norm.pdf(x, loc=p['birth'], scale=t)
            return y, x

        elif dim == 2:

            try:
                kernel = gaussian_kde(data[['birth', 'death']].values.T)
            except LinAlgError as e:
                print('Warning! Captured error %s'%e)
                print('Add one additional data point.')

                birth = data['birth'].subtract(1)
                death = data['death'].add(1)
                data = pd.concat((birth, death), axis=1)
                data = data.append(pd.DataFrame(dict(birth=np.max(data['birth']+1), death=0), index=[len(data)]))

                kernel = gaussian_kde(data[['birth', 'death']].values.T)

            if xmax is None:
                xmax = np.max(data['birth'].values)
            if xmin is None:
                xmin = np.min(data['birth'].values)
            if ymax is None:
                ymax = np.max(data['death'].values)
            if ymin is None:
                ymin = np.min(data['death'].values)

            X, Y = np.mgrid[xmin:xmax:np.complex(bins), ymin:ymax:np.complex(bins)]
            positions = np.vstack([X.ravel(), Y.ravel()])

            Z = np.reshape(kernel(positions).T, X.shape)
            return Z, [np.unique(X), np.unique(Y)]


m = LogitNet(random_state=17, **{"alpha":0.5, "standardize":0})
types = pd.DataFrame.from_csv('./data/types.csv').sort_values('c_num')

def classify_persistence(j, comb=('BC','DBC'), part='axon'):
    
    path = './data/'+part+'/'
    
    #get data 
    data_all = pd.read_csv(path+'persistence_truncated_%0.2f.csv'%(j*0.1))
    
    idx = (types['type'] == comb[0]) | (types['type'] == comb[1])
    data = pd.DataFrame()
    for c in types[idx]['c_num'].values:
        data = data.append(data_all[data_all['c_num'] == c])
   
    y = types[idx]['type'].values
    # shuffle labels for shuffled baseline
    y_shuffled = copy.copy(y)
    np.random.shuffle(y_shuffled)
    
    # get normalization boundaries and parameter for persistance image generation
    c_nums = np.unique(types[idx]['c_num'])
    max_birth = np.ceil(np.max(data_all['birth']))
    max_death = np.ceil(np.max(data_all['death']))
    
    min_birth = np.floor(np.min(data_all['birth']))
    min_death = np.floor(np.min(data_all['death']))
    dim=2
    bins=100
    X = np.zeros((len(c_nums),bins**dim))

    # generate persistence images
    for k,num in enumerate(c_nums):
        try:
            X[k,:] = get_persistence_image(data[data['c_num'] ==num], xmin=min_birth, xmax=max_birth, 
                                           ymin=min_death, ymax=max_death, dim=dim, bins=bins)[0].reshape(-1)
        except np.linalg.LinAlgError as e:
            return 'no image calculation possible anymore'
    
    X[X == np.nan] = 0
    X[X == np.inf] = 0
    X[X == -np.inf] = 0
    
    # Classification 
    pca = PCA(copy=True, whiten=False)
    kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=17)

    runs = []
    k = 1
    fm_lengths=np.array([0,X[0].shape[0]])
    # regular Classification
    for train_ix, test_ix in kf.split(X, y):

        X_train = X[train_ix]
        y_train = y[train_ix]
        X_test = X[test_ix]
        y_test = y[test_ix]
        
        try:
            X_train, X_test = get_pca_transformed_data(X_train, X_test, pca, fm_lengths)
        except ValueError as e:
            print(e)
            return 'no image calculation possible anymore'
        r_ = dict(truncated=j*10)
        r_['run_id'] = k

        m.fit(X_train, y_train)

        r_['accuracy_train'] = m.score(X_train, y_train)
        r_['accuracy_test'] = m.score(X_test, y_test)
        r_['log_loss_train'] = log_loss(y_train, m.predict_proba(X_train), labels=np.unique(y_train))
        r_['log_loss_test'] = log_loss(y_test, m.predict_proba(X_test), labels=np.unique(y_test))
        r_['shuffled'] = False
        runs.append(r_)
        k += 1
    k=1 
    # Classification on shuffled data
    for train_ix, test_ix in kf.split(X, y_shuffled):

        X_train = X[train_ix]
        y_train = y_shuffled[train_ix]
        X_test = X[test_ix]
        y_test = y_shuffled[test_ix]
        
        try:
            X_train, X_test = get_pca_transformed_data(X_train, X_test, pca, fm_lengths)
        except ValueError as e:
            print(e)
            return 'no image calculation possible anymore'
        r_ = dict(truncated=j*10)
        r_['run_id'] = k

        m.fit(X_train, y_train)

        r_['accuracy_train'] = m.score(X_train, y_train)
        r_['accuracy_test'] = m.score(X_test, y_test)
        r_['log_loss_train'] = log_loss(y_train, m.predict_proba(X_train), labels=np.unique(y_train))
        r_['log_loss_test'] = log_loss(y_test, m.predict_proba(X_test), labels=np.unique(y_test))
        r_['shuffled'] = True
        runs.append(r_)
        k += 1

    classification = pd.DataFrame(runs)
    save_path = './results/'+part+'/'+comb[0]+' vs '+comb[1]+'/'
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    classification.to_csv(save_path+'classification_persistence_truncated_%0.2f.csv'%(j*0.1))
    
    # train on entire data set and save the model and data
    try:
        X_, _ = get_pca_transformed_data(X,X,pca,fm_lengths)
    except ValueError as e:
        print(e)
        return 'no image calculation possible for all data'
    m.fit(X_,y)
    
    #open(save_path + 'classification_persistence_truncated_%0.2f_model'%(j*0.1),'wb').write(pickle.dumps(m))
    #open(save_path + 'classification_persistence_truncated_%0.2f_pca'%(j*0.1),'wb').write(pickle.dumps(pca))

In [98]:
truncations

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
for c in list(combinations(TYPES,2)):
    with multiprocessing.Pool(5) as pool:
        try:
            results = pool.map(partial(classify_persistence,comb=c,part='full'),truncations)
        except ValueError:
            continue

# Classify Density maps

In [9]:
def _project_data(dim, proj_axes, data):

    p_a = proj_axes
    if dim == 2:
        if len(p_a) < 2:
            print('Invalid parameter setting: The passed projection axes {0} do not \
                  fit with the dimension of the projection {1}'.format(p_a, dim))
        else:
            indices = '012'
            for ix in range(len(p_a)):
                indices = indices.replace(p_a[ix], '')
            deleted_axis = int(indices)
            ax = [0, 1, 2]
            ax.remove(deleted_axis)
            result = data[:, ax]

    elif dim == 1:
        if len(p_a) > 1:
            print('Invalid parameter setting: The passed projection axes {0} do not \
                  fit with the dimension of the projection {1}'.format(p_a, dim))
            
        else:
            ax = int(p_a)
            result = data[:, ax]
    else:
        result = data

    return result

from utils.utils import smooth_gaussian
def get_density_map(pc,proj_axes, r):
    

    dim = len(proj_axes)
    pc = (pc - r['min'][0]) / (r['max'][0] - r['min'][0])

    data = _project_data(dim, proj_axes, pc)

    range_ = [[-.1, 1.1]] * dim
    H, edges = np.histogramdd(data, bins=(100,) * dim,
                              range=range_, normed=True)

    H = smooth_gaussian(H, dim=dim, sigma=2)
    return H


m = LogitNet(random_state=17, **{"alpha":0.5, "standardize":0})

def classify_density_map(j, comb=('BC','DBC'), part='axon'):
    
    path = './data/'+part+'/'
    
    #get data 
    data_all = pd.read_csv(path+'point_cloud_truncated_%0.2f.csv'%(j*0.1))
    
    idx = (types['type'] == comb[0]) | (types['type'] == comb[1])
    data = pd.DataFrame()
    for c in types[idx]['c_num'].values:
        data = data.append(data_all[data_all['c_num'] == c])
   
    y = types[idx]['type'].values
    y_shuffled = copy.copy(y)
    np.random.shuffle(y_shuffled)
    
    c_nums = np.unique(types[idx]['c_num'])
    r = dict(min=np.min(data_all[['x','y','z']]).values, max=np.max(data_all[['x','y','z']]).values)
    X = np.zeros((len(c_nums),10000))

    for k,num in enumerate(c_nums):
        
        D = data[data['c_num'] ==num][['x','y','z']].values
        X[k,:] = get_density_map(D, '02', r).reshape(-1)

    X[X == np.nan] = 0
    X[X == np.inf] = 0
    X[X == -np.inf] = 0
    
    
    #### CLASSIFICATION ######
    pca = PCA(copy=True, whiten=False)    
    kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=17)

    runs = []
    k = 1
    fm_lengths=np.array([0,X[0].shape[0]])
    for train_ix, test_ix in kf.split(X, y):

        X_train = X[train_ix]
        y_train = y[train_ix]
        X_test = X[test_ix]
        y_test = y[test_ix]
        
        try:
            X_train, X_test = get_pca_transformed_data(X_train, X_test, pca, fm_lengths)
        except ValueError as e:
            print(e)
            return 'no image calculation possible anymore'
        r_ = dict(truncated=j*10)
        r_['run_id'] = k

        m.fit(X_train, y_train)

        r_['accuracy_train'] = m.score(X_train, y_train)
        r_['accuracy_test'] = m.score(X_test, y_test)
        r_['log_loss_train'] = log_loss(y_train, m.predict_proba(X_train), labels=np.unique(y_train))
        r_['log_loss_test'] = log_loss(y_test, m.predict_proba(X_test), labels=np.unique(y_test))
        r_['shuffled'] = False
        runs.append(r_)
        k += 1
    k=1 
    for train_ix, test_ix in kf.split(X, y_shuffled):

        X_train = X[train_ix]
        y_train = y_shuffled[train_ix]
        X_test = X[test_ix]
        y_test = y_shuffled[test_ix]
        
        try:
            X_train, X_test = get_pca_transformed_data(X_train, X_test, pca, fm_lengths)
        except ValueError as e:
            print(e)
            return 'no image calculation possible anymore'
        r_ = dict(truncated=j*10)
        r_['run_id'] = k

        m.fit(X_train, y_train)

        r_['accuracy_train'] = m.score(X_train, y_train)
        r_['accuracy_test'] = m.score(X_test, y_test)
        r_['log_loss_train'] = log_loss(y_train, m.predict_proba(X_train), labels=np.unique(y_train))
        r_['log_loss_test'] = log_loss(y_test, m.predict_proba(X_test), labels=np.unique(y_test))
        r_['shuffled'] = True
        runs.append(r_)
        k += 1

    classification = pd.DataFrame(runs)
    save_path = './results/'+part+'/'+comb[0]+' vs '+comb[1]+'/'
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    classification.to_csv(save_path+'classification_density_map_truncated_%0.2f.csv'%(j*0.1))
    
    # train on entire data set and save the model and data
    try:
        X_, _ = get_pca_transformed_data(X,X,pca,fm_lengths)
    except ValueError as e:
        print(e)
        return 'no image calculation possible for all data'
    m.fit(X_,y)
    
    #open(save_path + 'classification_density_map_truncated_%0.2f_model'%(j*0.1),'wb').write(pickle.dumps(m))
    #open(save_path + 'classification_density_map_truncated_%0.2f_pca'%(j*0.1),'wb').write(pickle.dumps(pca))

In [10]:
for c in list(combinations(TYPES,2)):
    with multiprocessing.Pool(5) as pool:
        try:
            results = pool.map(partial(classify_density_map,comb=c,part='full'),truncations)
        except ValueError:
            continue